In [8]:
import random

def roll_dice(num_dice=5):
    """
    Simulate rolling dice.

    Parameters:
        num_dice (int): Number of dice to roll. Default is 5.

    Returns:
        list: List containing the values of the rolled dice.
    """
    return [random.randint(1, 6) for _ in range(num_dice)]

def display_dice(dice_values):
    """
    Display the current dice values.

    Parameters:
        dice_values (list): List containing the values of the dice.
    """
    print("Current dice values:")
    for i, value in enumerate(dice_values):
        print(f"Die {i+1}: {value}")


In [14]:
def choose_dice_to_reroll():
    """
    Allow the user to input the indices of the dice they want to reroll.

    Returns:
        list: List containing the indices of the dice to reroll (0-indexed).
    """
    while True:
        try:
            user_input = input("Enter the indices of the dice you want to reroll (separated by spaces), or 'done' if you want to keep the current dice values: ")
            if user_input.lower() == 'done':
                return []  # Return an empty list if the user wants to keep the current dice values
            else:
                indices = [int(idx) - 1 for idx in user_input.split()]
                if not all(0 <= idx <= 4 for idx in indices):
                    raise ValueError("Indices must be between 1 and 5")
                return indices
        except ValueError as e:
            print("Invalid input. Please enter valid indices between 1 and 5.")


In [11]:
def reroll_selected_dice(dice_values, indices_to_reroll):
    """
    Reroll the selected dice and update their values.

    Parameters:
        dice_values (list): List containing the current values of the dice.
        indices_to_reroll (list): List containing the indices of the dice to reroll (0-indexed).

    Returns:
        list: List containing the updated values of the dice.
    """
    for idx in indices_to_reroll:
        dice_values[idx] = random.randint(1, 6)
    return dice_values


In [10]:
def reroll_dice(dice_values):
    """
    Allow the user to choose which dice to reroll, reroll them, and display the combined result.

    Parameters:
        dice_values (list): List containing the current values of the dice.

    Returns:
        list: List containing the updated values of the dice after rerolling.
    """
    indices_to_reroll = choose_dice_to_reroll()
    if indices_to_reroll:
        new_dice_values = reroll_selected_dice(dice_values, indices_to_reroll)
        combined_dice_values = [new_dice_values[i] if i in indices_to_reroll else dice_values[i] for i in range(len(dice_values))]
        print("Combined dice values after reroll:", combined_dice_values)
        return combined_dice_values
    else:
        print("Keeping current dice values.")
        return dice_values


In [12]:
def calculate_score(dice_values):
    """
    Calculate the score for all categories based on the current dice values.

    Parameters:
        dice_values (list): List containing the current values of the dice.

    Returns:
        dict: Dictionary containing the score for each category.
    """
    score = {}

    # Upper section (Ones to Sixes)
    for i in range(1, 7):
        score[f"{i}s"] = sum(dice for dice in dice_values if dice == i)

    # Three of a kind
    if any(dice_values.count(dice) >= 3 for dice in dice_values):
        score["Three of a Kind"] = sum(dice_values)
    else:
        score["Three of a Kind"] = 0

    # Four of a kind
    if any(dice_values.count(dice) >= 4 for dice in dice_values):
        score["Four of a Kind"] = sum(dice_values)
    else:
        score["Four of a Kind"] = 0

    # Full House
    if len(set(dice_values)) == 2 and any(dice_values.count(dice) == 3 for dice in set(dice_values)):
        score["Full House"] = 25
    else:
        score["Full House"] = 0

    # Small Straight
    if {1, 2, 3, 4}.issubset(set(dice_values)) or {2, 3, 4, 5}.issubset(set(dice_values)) or {3, 4, 5, 6}.issubset(set(dice_values)):
        score["Small Straight"] = 30
    else:
        score["Small Straight"] = 0

    # Large Straight
    if {1, 2, 3, 4, 5} == set(dice_values) or {2, 3, 4, 5, 6} == set(dice_values):
        score["Large Straight"] = 40
    else:
        score["Large Straight"] = 0

    # Yahtzee
    if dice_values.count(dice_values[0]) == 5:
        score["Yahtzee"] = 50
    else:
        score["Yahtzee"] = 0

    # Chance
    score["Chance"] = sum(dice_values)

    return score


In [20]:
def main():
    print("Welcome to Yahtzee!")

    # Roll the initial set of dice
    print("\nFirst roll:")
    dice_values = roll_dice()
    display_dice(dice_values)

    # reroll
    for _ in range(2):
        reroll_choice = input("\nWould you like to reroll any dice? (yes/no): ").lower()
        if reroll_choice == 'yes':
            dice_values = reroll_dice(dice_values)
            display_dice(dice_values)
        else:
            break


    # Calculate the final score
    score = calculate_score(dice_values)
    print("\nFinal score:")
    for category, category_score in score.items():
        print(f"{category}: {category_score}")


if __name__ == "__main__":
    main()


Welcome to Yahtzee!

First roll:
Current dice values:
Die 1: 6
Die 2: 4
Die 3: 3
Die 4: 5
Die 5: 4

Would you like to reroll any dice? (yes/no): yes
Enter the indices of the dice you want to reroll (separated by spaces), or 'done' if you want to keep the current dice values: 3 4 
Combined dice values after reroll: [6, 4, 6, 4, 4]
Current dice values:
Die 1: 6
Die 2: 4
Die 3: 6
Die 4: 4
Die 5: 4

Would you like to reroll any dice? (yes/no): no

Final score:
1s: 0
2s: 0
3s: 0
4s: 12
5s: 0
6s: 12
Three of a Kind: 24
Four of a Kind: 0
Full House: 25
Small Straight: 0
Large Straight: 0
Yahtzee: 0
Chance: 24


In [17]:
if __name__ == "__main__":
    main()



Welcome to Yahtzee!

First roll:
Current dice values:
Die 1: 6
Die 2: 6
Die 3: 2
Die 4: 2
Die 5: 2

Would you like to reroll any dice? (yes/no): no

Final score:
1s: 0
2s: 6
3s: 0
4s: 0
5s: 0
6s: 12
Three of a Kind: 18
Four of a Kind: 0
Full House: 25
Small Straight: 0
Large Straight: 0
Yahtzee: 0
Chance: 18


In [21]:
if __name__ == "__main__":
    main()

Welcome to Yahtzee!

First roll:
Current dice values:
Die 1: 3
Die 2: 6
Die 3: 3
Die 4: 6
Die 5: 3

Would you like to reroll any dice? (yes/no): yes
Enter the indices of the dice you want to reroll (separated by spaces), or 'done' if you want to keep the current dice values: 1 3 5 
Combined dice values after reroll: [5, 6, 5, 6, 3]
Current dice values:
Die 1: 5
Die 2: 6
Die 3: 5
Die 4: 6
Die 5: 3

Would you like to reroll any dice? (yes/no): yes
Enter the indices of the dice you want to reroll (separated by spaces), or 'done' if you want to keep the current dice values: 5 
Combined dice values after reroll: [5, 6, 5, 6, 6]
Current dice values:
Die 1: 5
Die 2: 6
Die 3: 5
Die 4: 6
Die 5: 6

Final score:
1s: 0
2s: 0
3s: 0
4s: 0
5s: 10
6s: 18
Three of a Kind: 28
Four of a Kind: 0
Full House: 25
Small Straight: 0
Large Straight: 0
Yahtzee: 0
Chance: 28
